# annotationComparison.ipynb
### Marcus Viscardi,    January 28, 2025

In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
current_dir = Path.cwd()
old_gtf = current_dir / "Caenorhabditis_elegans.WBcel235.100.gtf"
new_txt = current_dir / "c_elegans.PRJNA13758.WS295.geneIDs.txt"
assert old_gtf.exists(), f"old_gtf does not exist: {old_gtf}"
assert new_txt.exists(), f"new_gff does not exist: {new_txt}"

In [ ]:
new = pd.read_csv(new_txt, header=None)[[1, 2, 3, 5]].rename(columns={1: "gene_id", 2: "gene_name", 3:"contig", 5: "gene_biotype"})
new['gene_name'] = new['gene_name'].str.replace(" ", "")
# I want to fill the contig column with the gene_name if it is missing
new['gene_name'] = new['gene_name'].fillna(new['contig'])
new

In [ ]:
old = pd.read_csv(old_gtf, sep="\t", comment="#", header=None)
# print(old.head())
old = old[[0, 3, 4, 2, 8]].rename(columns={2: "feature", 8: "gene_info", 0: "chr", 3: "start", 4: "end"}).query('feature == "gene"')
old[['gene_id', 'gene_name', 'gene_biotype']] = old['gene_info'].str.extract(r'gene_id "(.*?)";.*?gene_name "(.*?)";.*?gene_biotype "(.*?)";')
# print(old.columns)
old = old[['gene_id', 'gene_name', 'gene_biotype', 'chr', "start", "end"]]
old

In [ ]:
merge = old.merge(new, on="gene_id", how="outer", suffixes=["_old", "_new"], indicator=True)
print(merge['_merge'].value_counts())
merge['gene_name_changed'] = (merge['gene_name_old'] != merge['gene_name_new']) & (~merge['gene_name_old'].isna()) & (~merge['gene_name_new'].isna())
merge.query("gene_name_changed")[["gene_id", "gene_name_old", "gene_name_new"]].to_csv(current_dir / "allChangedGeneNames_WBcel235-to-WS295.csv", index=False)
merge

In [ ]:
old_tableS3 = current_dir / "240906_tableS3_totalMerge_fisherTestedOnly - 230927_totalMerge_fisherTestedOnly.csv"
hits = pd.read_csv(old_tableS3)
hits

In [ ]:
hits_merge = hits.merge(merge[["gene_id", "gene_name_new"]], on="gene_id", how="left", suffixes=["_hits", "_annots"])
# hits_merge['_merge'].value_counts()
hits_merge.query("gene_name != gene_name_new")[['gene_id', 'gene_name', 'gene_name_new']]
hits_merge[["gene_id", "gene_name_new", "gene_name"]].sort_values("gene_id").to_csv(current_dir / "gene_name_changes.csv", index=False)

In [ ]:
other_genes_to_check_str = "rpl-30, rps-15A, rps-27A, rpl-7A, rpl-3, rpl-10A, rpl-12, hel-1, aly-3, rsp-6, K08D12.3, R06C1.4, C53H9.2, rsp-5, ZK228.4, rpl-26, pqn-70, smd-1, rpl-10A, C45B2.8, farl-11, odc-1, zip-12, Y73B3A.18, rpl-7A, rpl-30, F19B2.5, nhr-114, C30E1.9, rsp-6, tos-1, rps-27A, C53H9.2, rpl-3, T05E12.6, H28G03.2, C35B1.2, ddo-2, R06C1.4, rps-15A, col-182, Y39B6A.21, rps-27A, rps-15A, rpl-30, rpl-12, rpl-3, C53H9.2, K08D12.3, rpl-26, rpl-7A"
other_genes_to_check = list(set(sorted(other_genes_to_check_str.split(", "))))
for gene_to_check in other_genes_to_check:
    res = merge.query("gene_name_old == @gene_to_check or gene_name_new == @gene_to_check")[['gene_id', 'gene_name_old', 'gene_name_new']]
    if not res.empty:
        old_name = res['gene_name_old'].values[0]
        new_name = res['gene_name_new'].values[0]
        if old_name != new_name:
            print(f"{gene_to_check} found in changed names list: {old_name} -> {new_name}")
        else:
            print(f"{gene_to_check} unchanged.")
    else:
        print(f"{gene_to_check} not found in changed names list.")
merge.query("gene_name_old in @other_genes_to_check or gene_name_new in @other_genes_to_check")[['gene_id', 'gene_name_old', 'gene_name_new', 'gene_name_changed']]